# Compare Two json file and add missing keys to next json. user this method to get updated json file for localization in GEC

In [3]:
import json
from collections import OrderedDict
import os

def merge_dicts(d1, d2):
    result = {}
    for key in d1.keys() | d2.keys():
        if key in d1 and key in d2:
            if isinstance(d1[key], dict) and isinstance(d2[key], dict):
                result[key] = merge_dicts(d1[key], d2[key])
            else:
                result[key] = d2[key]
        elif key in d1:
            result[key] = d1[key]
        else:
            result[key] = d2[key]
    return result

def find_missing_keys(d1, d2):
    missing_keys = {}
    for key, value in d2.items():
        if isinstance(value, dict):
            # Recursively find missing keys in nested dictionaries
            missing_keys[key] = find_missing_keys(d1.setdefault(key, {}), value)
        elif key not in d1:
            # Add missing key to the dictionary with value of null
            # translated = translate_to(value, 'japanese')
            translated = value
            d1[key] = translated
            missing_keys[key] = translated
    return missing_keys

def write_missing_keys(file1_path, file2_path, output_file_path):
    with open(file1_path, 'r', encoding='utf-8') as file1:
        d1 = json.load(file1)
    with open(file2_path, 'r', encoding='utf-8') as file2:
        d2 = json.load(file2)
    
    missing_keys = find_missing_keys(d1, d2)
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(missing_keys, output_file, indent=4, ensure_ascii=False)


def sort_json(obj):
    if isinstance(obj, dict):
        return OrderedDict(sorted((k, sort_json(v)) for k, v in obj.items()))
    if isinstance(obj, list):
        return sorted(sort_json(x) for x in obj)
    elif obj is not None:  # Add this condition to handle None values
        return obj
    else:
        return ""

def merge_and_sort(file1_path, missing_file_path, output_file_path):
    with open(file1_path, 'r', encoding='utf-8') as file1:
        data1 = json.load(file1)

    with open(missing_file_path, 'r', encoding='utf-8') as missing_file:
        missing_data = json.load(missing_file)
    
    merged_data = merge_dicts(data1, missing_data)
    sorted_data = sort_json(merged_data)
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(sorted_data, output_file, indent=4, ensure_ascii=False)
        

In [10]:
# Paths to the JSON files
# add missing keys from file 1 to file 2
file1_path = '/Users/sanjokdangol/Projects/gec/gec-frontend/src/libs/i18n/locales/vn.json'
file2_path = '/Users/sanjokdangol/Projects/gec/gec-frontend/src/libs/i18n/locales/en.json'

filename = os.path.basename(file1_path)

output_file_path = '/Users/sanjokdangol/Projects/jupyternote/Translator/translated_json/'+filename+'.json'
final_output_file_path = '/Users/sanjokdangol/Projects/jupyternote/Translator/translated_json/'+filename+'.json'


# Write the missing keys from file 2 into a new file
write_missing_keys(file1_path, file2_path, output_file_path)
merge_and_sort(file1_path, output_file_path, final_output_file_path)

# Print confirmation message
print(f"Missing keys from file 2 have been merged and sorted into: {final_output_file_path}")


Missing keys from file 2 have been merged and sorted into: /Users/sanjokdangol/Projects/jupyternote/Translator/translated_json/vn.json.json
